# Aprendizado Federado para reconhecimento de contexto em dispositivos móveis
## Experimento 1 - Pré-treinamento usual com uma base de usuários e Aprendizado Federado embarcado partindo do modelo pretreinado 

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import es_utils as utils

Inicialmente precisamos separar quais 'labels' usaremos. Aqui, optamos pelas 51 labels que segundo artigo do ExtraSensosy utiliza, de modo a ter bons parâmetros de comparação

In [2]:
labels = ['label:LYING_DOWN',
 'label:SITTING',
 'label:FIX_walking',
 'label:FIX_running',
 'label:BICYCLING',
 'label:SLEEPING',
 'label:LAB_WORK',
 'label:IN_CLASS',
 'label:IN_A_MEETING',
 'label:LOC_main_workplace',
 'label:OR_indoors',
 'label:OR_outside',
 'label:IN_A_CAR',
 'label:ON_A_BUS',
 'label:DRIVE_-_I_M_THE_DRIVER',
 'label:DRIVE_-_I_M_A_PASSENGER',
 'label:LOC_home',
 'label:FIX_restaurant',
 'label:PHONE_IN_POCKET',
 'label:OR_exercise',
 'label:COOKING',
 'label:SHOPPING',
 'label:STROLLING',
 'label:DRINKING__ALCOHOL_',
 'label:BATHING_-_SHOWER',
 'label:CLEANING',
 'label:DOING_LAUNDRY',
 'label:WASHING_DISHES',
 'label:WATCHING_TV',
 'label:SURFING_THE_INTERNET',
 'label:AT_A_PARTY',
 'label:AT_A_BAR',
 'label:LOC_beach',
 'label:SINGING',
 'label:TALKING',
 'label:COMPUTER_WORK',
 'label:EATING',
 'label:TOILET',
 'label:GROOMING',
 'label:DRESSING',
 'label:AT_THE_GYM',
 'label:STAIRS_-_GOING_UP',
 'label:STAIRS_-_GOING_DOWN',
 'label:ELEVATOR',
 'label:OR_standing',
 'label:AT_SCHOOL',
 'label:PHONE_IN_HAND',
 'label:PHONE_IN_BAG',
 'label:PHONE_ON_TABLE',
 'label:WITH_CO-WORKERS',
 'label:WITH_FRIENDS']

len(labels)

51

Separamos os 60 usuários em 5 pastas onde cada pasta considera 40 usuários selecionados aleatoriamente para treino e os 20 restantes para teste. Os dados dos 20 usuários são ainda separados em 4 arquivos *.csv* (x_train, x_test, y_train, y_test) e colocados em pastas nomeadas com o uuid do usuário.

In [4]:
#k_folds = utils.create_k_folds_n_users(5, 3, '/home/wander/OtherProjects/har_flower/sample_data')
k_folds = utils.create_k_folds_n_users(5, 40, '/home/wander/OtherProjects/har_flower/full_data')

Em seguida, para cada pasta, treinamos um modelo de MLP fazendo uma otimização de hiperparâmetros e convertemos em um modelo *tflite*. Os hiperparâmetros otimizados são:

- número de camadas escondidas (*hidden layers*): 1 ou 2
- número de neurônios em cada camada escondida: 4 a 64, com um passo de tamanho 4
- constante de penalização da regularização L2 em cada camada: 1e-2, 1e-3 ou 1e-4
- taxa de aprendizado (*learning rate*) do otimizador (*Stochastic Gradient Descent*, **SGD**): 1e-1, 1e-2, 1e-3 ou 1e-4
- momentum do otimizador (**SGD**): 0.8, 0.5, 0.3 ou 0.1

In [5]:

for fold in k_folds:
    config = {'df': k_folds[fold]['train']}
    har = utils.HAR(config)
    #har.run()
    har.hypertunning()
    
    har.mlp.model.save(f'model/saved_model_fold{fold}')
    converter = tf.lite.TFLiteConverter.from_saved_model(f'model/saved_model_fold{fold}') # path to the SavedModel directory
    tflite_model = converter.convert()
    with open(f'model/model_base_{fold}.tflite', 'wb') as f:
      f.write(tflite_model)

INFO:tensorflow:Reloading Oracle from existing project hypertunning/experimento_1/oracle.json
INFO:tensorflow:Reloading Tuner from hypertunning/experimento_1/tuner0.json
INFO:tensorflow:Oracle triggered exit


2022-03-17 15:05:20.081426: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-17 15:05:20.163007: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-03-17 15:05:20.163023: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-03-17 15:05:20.164006: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

Epoch 1/50
3719/3719 [==============================] - 4s 864us/step - loss: 0.3322 - categorical_accuracy: 0.7099 - val_loss: 0.3202 - val_categorical_accuracy: 0.7212
Epoch 2/50
3719/3719 [==============================] - 3s 785us/step - loss: 0.3185 - categorical_accuracy: 0.7213 - val_loss: 0.3157 - val_categorical_accuracy: 0.7249
Epoch 3/50
3719/3719 [==============================] - 3s 826us/step - loss: 0.3155 - categorical_accuracy: 0.7238 - val_loss: 0.3137 - val_categorical_accuracy: 0.7268
Epoch 4/50
3719/3719 [==============================] - 3s 823us/step - loss: 0.3138 - categorical_accuracy: 0.7249 - val_loss: 0.3123 - val_categorical_accuracy: 0.7276
Epoch 5/50
3719/3719 [==============================] - 3s 855us/step - loss: 0.3126 - categorical_accuracy: 0.7259 - val_loss: 0.3112 - val_categorical_accuracy: 0.7283
Epoch 6/50
3719/3719 [==============================] - 3s 838us/step - loss: 0.3117 - categorical_accuracy: 0.7265 - val_loss: 0.3106 - val_categoric

Epoch 49/50
3719/3719 [==============================] - 3s 779us/step - loss: 0.3037 - categorical_accuracy: 0.7329 - val_loss: 0.3028 - val_categorical_accuracy: 0.7338
Epoch 50/50
3719/3719 [==============================] - 3s 814us/step - loss: 0.3036 - categorical_accuracy: 0.7330 - val_loss: 0.3027 - val_categorical_accuracy: 0.7339
Best epoch: 50


2022-03-17 15:07:51.339585: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 216093104 exceeds 10% of free system memory.


...

INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/50
4114/4114 [==============================] - 4s 841us/step - loss: 0.3110 - categorical_accuracy: 0.7288 - val_loss: 0.2995 - val_categorical_accuracy: 0.7383
Epoch 2/50
4114/4114 [==============================] - 3s 774us/step - loss: 0.2987 - categorical_accuracy: 0.7378 - val_loss: 0.2962 - val_categorical_accuracy: 0.7400
Epoch 3/50
4114/4114 [==============================] - 3s 814us/step - loss: 0.2961 - categorical_accuracy: 0.7394 - val_loss: 0.2944 - val_categorical_accuracy: 0.7414
Epoch 4/50
4114/4114 [==============================] - 3s 811us/step - loss: 0.2946 - categorical_accuracy: 0.7407 - val_loss: 0.2932 - val_categorical_accuracy: 0.7430
Epoch 5/50
4114/4114 [==============================] - 3s 795us/step - loss: 0.2935 - categorical_accuracy: 0.7415 - val_loss: 0.2925 - val_categorical_accuracy: 0.7432
Epoch 6/50
4114/4114 [==============================] - 3s 826us/step - loss: 0.2927 - categorical_accuracy: 0.7421 - val_loss: 0.2916 - val_categoric

Epoch 49/50
4114/4114 [==============================] - 3s 780us/step - loss: 0.2853 - categorical_accuracy: 0.7480 - val_loss: 0.2848 - val_categorical_accuracy: 0.7480
Epoch 50/50
4114/4114 [==============================] - 3s 783us/step - loss: 0.2853 - categorical_accuracy: 0.7480 - val_loss: 0.2848 - val_categorical_accuracy: 0.7480
Best epoch: 42
Epoch 1/42
13162/13162 [==============================] - 10s 738us/step - loss: 0.3031 - categorical_accuracy: 0.7353 - val_loss: 0.2951 - val_categorical_accuracy: 0.7420
Epoch 2/42
13162/13162 [==============================] - 10s 739us/step - loss: 0.2941 - categorical_accuracy: 0.7411 - val_loss: 0.2920 - val_categorical_accuracy: 0.7423
Epoch 3/42
13162/13162 [==============================] - 10s 742us/step - loss: 0.2919 - categorical_accuracy: 0.7421 - val_loss: 0.2902 - val_categorical_accuracy: 0.7444
Epoch 4/42
13162/13162 [==============================] - 10s 746us/step - loss: 0.2905 - categorical_accuracy: 0.7432 - val

INFO:tensorflow:Assets written to: model/saved_model/assets
2022-03-17 15:55:26.429419: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-03-17 15:55:26.429438: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-03-17 15:55:26.429441: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:372] Ignored change_concat_input_ranges.
2022-03-17 15:55:26.429626: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: model/saved_model
2022-03-17 15:55:26.430541: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-03-17 15:55:26.430553: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: model/saved_model
2022-03-17 15:55:26.433074: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-03-17 15:55:26.457030: I tensorflow/cc/saved_model/loader.cc:212] Running initi

Com os modelos salvos, o restante do experimento segue em ambiente Android